# Are you a fan of Mayor Han?

<img src="https://i.imgur.com/GgPPfk4.png" width="900" />

(image source: [link](https://www.storm.mg/article/525105?srcid=gAAAAABcsea_WZX1btdtDfPbAuBP8p_-9GcnyFvKDxXCZIjJg9dNW26Gskj7oZGDlmsQQjZQq_6v0Pqx57FbQhbMy7_D2Mveyf8pecA6NS_0Kxhj9N1P1CI%253D))

## 1. Define Question
Recently, Mayor of Kaohsiung Han Kuo-yu (韓國瑜) is in fasion, and even his name became a buzzword.
In this section, I'll **determine whether a person is a fan of Mayor Han**.

## 2. Generate mock dataset 

This dataset contains 20000 rows. The attributes are:
- **gender**: male/female
- **age**: int
- **party**: KMT/Democratic Progressive Party
- **hasMCTfamily**: 0/1
    - if any of your family works as a military personnel, a civil servant or a teacher
- **loveFerrisWheel**: 0/1
- **loveChina**: 0/1
- **willToReunification**: 0%~100%

and the label indicates whether this person loves mayor Han or not:
- **fanOfHan**: 0/1

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import random
import math

### gender
- according to [wikipedia](https://zh.wikipedia.org/wiki/%E8%87%BA%E7%81%A3%E4%BA%BA%E5%8F%A3#%E4%BA%BA%E5%8F%A3), the average ratio of male and femail is about 0.99

In [2]:
def decideGenger(n):
    return 'male' if n*random.uniform(0, 1) < 0.5 else 'female'

pdata = pd.DataFrame(1, index=range(0,20000), columns=['gender'])
pdata['gender'] = pdata['gender'].apply(decideGenger)
pdata.head()

,gender
0,female
1,male
2,female
3,female
4,female


### age
- again, according to [wikipedia](https://zh.wikipedia.org/wiki/%E8%87%BA%E7%81%A3%E4%BA%BA%E5%8F%A3#%E4%BA%BA%E5%8F%A3), we can consider this a normal distribution
- I simply set mean = 50, and std,dev = 16

In [3]:
# pdata['age'] = 0
def decideAge(n):
    if n < 15:
        return 20
    elif n > 90:
        return 85

mu, sigma = 50, 15 # mean and standard deviation
pdata['age'] = np.random.normal(mu, sigma, 20000)
pdata['age'] = pdata['age'].apply(round)
pdata.head()

,gender,age
0,female,31
1,male,58
2,female,68
3,female,77
4,female,36


### party
- from the research [1992/06~2018/12](https://esc.nccu.edu.tw/app/news.php?Sn=165#), we can tell the party preference distribution.
- random_percent = random(0, 1.0)
- if random_percent <= 0.246, then 'Democratic Progressive Party'
- else if 0.246 < random_percent <= 0.561, then 'KMT'
- else 'none'
![](https://i.imgur.com/2XfwH3V.png)
(image source: [link](https://esc.nccu.edu.tw/app/news.php?Sn=165#))

In [4]:
def decideParty(n):
    n = n*random.uniform(0, 1)
    if n <= 0.246:
        return 'Democratic Progressive Party'
    elif n > 0.246 and n <= 0.561:
        return 'KMT'
    else:
        return 'none'

pdata['party'] = 1
pdata['party'] = pdata['party'].apply(decideParty)
pdata.head()

,gender,age,party
0,female,31,Democratic Progressive Party
1,male,58,Democratic Progressive Party
2,female,68,Democratic Progressive Party
3,female,77,none
4,female,36,KMT


### willToReunification
- one's will to reunification with China
- if party == 'KMT', then there's 40%~100% of chance
- else 0%~50%. I believe that no one wants this to happen

In [5]:
pdata['willToReunification'] = np.nan
for index, row in pdata.iterrows():
    pdata.at[index,'willToReunification'] = round(random.uniform(0.6, 1), 2) if row['party'] == 'KMT' else round(random.uniform(0, 1), 2)

pdata.head()

,gender,age,party,willToReunification
0,female,31,Democratic Progressive Party,0.04
1,male,58,Democratic Progressive Party,0.98
2,female,68,Democratic Progressive Party,0.35
3,female,77,none,0.48
4,female,36,KMT,0.75


### hasMCTfamily
- if one's family works as a **military personnel**, a **civil servant** or a **teacher** (軍公教人員), there's a great chance that he/she won't like Democratic Progressive Party, because of the "pension reform (年金改革)" policy, thus, he/she might be forced to support Han in order not to let Chi-Mai Chen, who stands for Democratic Progressive Party, win the election.
- from [this report](http://www.fund.gov.tw/public/data/6851558871.pdf), I found that around 3% of population works as a military personnel, a civil servant or a teacher. and let's assume all their family feel sad for them and thus decide to select Mayor Han, with each family 6 people. That's total 18% approximately.
- randomly choose 18% from dataset

In [6]:
def decideMCTfamily(_):
    n = random.uniform(0, 1)
    return 1 if n <= 0.18 else 0

pdata['hasMCTfamily'] = 1
pdata['hasMCTfamily'] = pdata['hasMCTfamily'].apply(decideMCTfamily)
pdata.head()

,gender,age,party,willToReunification,hasMCTfamily
0,female,31,Democratic Progressive Party,0.04,0
1,male,58,Democratic Progressive Party,0.98,0
2,female,68,Democratic Progressive Party,0.35,0
3,female,77,none,0.48,0
4,female,36,KMT,0.75,0


### loveChina
- it depends, but if one's prefered party is KMT, he/she 's more likely (70%) to love China
- if party == 'KMT' and random_percent >= 0.3, then 1 
- else 0

In [7]:
pdata['loveChina'] = 1
for index, row in pdata.iterrows():
    pdata.at[index,'loveChina'] = 1 if row['party'] == 'KMT' and random.uniform(0, 1) > 0.3 else 0

pdata.head()

,gender,age,party,willToReunification,hasMCTfamily,loveChina
0,female,31,Democratic Progressive Party,0.04,0,0
1,male,58,Democratic Progressive Party,0.98,0,0
2,female,68,Democratic Progressive Party,0.35,0,0
3,female,77,none,0.48,0,0
4,female,36,KMT,0.75,0,0


### loveFerrisWheel
- haters' gonna hate.
- randomly selected (50%)

In [8]:
def decideLoveFerrisWheel(_):
    return 1 if random.uniform(0, 1) < 0.5 else 0

pdata['loveFerrisWheel'] = 1
pdata['loveFerrisWheel'] = pdata['loveFerrisWheel'].apply(decideLoveFerrisWheel)
pdata.head()

,gender,age,party,willToReunification,hasMCTfamily,loveChina,loveFerrisWheel
0,female,31,Democratic Progressive Party,0.04,0,0,0
1,male,58,Democratic Progressive Party,0.98,0,0,1
2,female,68,Democratic Progressive Party,0.35,0,0,0
3,female,77,none,0.48,0,0,1
4,female,36,KMT,0.75,0,0,1


## 3. Define rules:

<img src="" width="900" />

### fanOfHan
Now I'm gonna label the dataset, and the following are the rules.
1. if one's party is KMT => fan of Mayor Han
2. if one's willing to reunification with China over 70% => fan of Mayor Han
3. if one match 3 or more rules listed below => fan of Mayor Han
    - loveChina = 1
    - hasMCTfamily = 1
    - loveFerrisWheel = 1
    - gender = 'Female' and age >= 55

In [9]:
pdata['fanOfHan'] = 0
for index, row in pdata.iterrows():
    if (row['party'] == 'KMT') or (row['willToReunification'] >= 0.9):
        pdata.at[index,'fanOfHan'] = 1
        continue
    if (row['gender'] == 'female') and (row['age'] >= 55):
        pdata.at[index,'fanOfHan'] = 1
        continue
        
    match = 0
    match = match + row['loveChina'] + row['hasMCTfamily'] + row['loveFerrisWheel']

    pdata.at[index,'fanOfHan'] = 1 if match > 1 else 0

# save the dataset as csv
pdata.to_csv('mock_data.csv')
pdata.head()

,gender,age,party,willToReunification,hasMCTfamily,loveChina,loveFerrisWheel,fanOfHan
0,female,31,Democratic Progressive Party,0.04,0,0,0,0
1,male,58,Democratic Progressive Party,0.98,0,0,1,1
2,female,68,Democratic Progressive Party,0.35,0,0,0,1
3,female,77,none,0.48,0,0,1,1
4,female,36,KMT,0.75,0,0,1,1


## 4. Construct Decision Tree with tools:

In [10]:
# from scipy.stats import logistic
# def sigmoid(x):
#     return 1 / (1 + math.exp(-x))

# pdata['fanOfHan'] = 0
# for index, row in pdata.iterrows():
#     isAgedFemale = 1 if row['gender'] == 'Female' and row['age'] > 50 else 0
#     isKMT = 1 if row['party'] == 'KMT' else 0
#     percentage = (isAgedFemale*0.1) + (isKMT*0.99) + (row['hasMCTfamily']*0.8) + (row['loveFerrisWheel']*0.7) + (row['loveChina']*0.5) + (row['wantToReunification']*0.99)
# #     percentage = logistic.cdf(percentage)
#     percentage = sigmoid(percentage)
# #     print(percentage)
#     pdata.at[index,'fanOfHan'] = 1 if percentage > 0.9 else 0

# pdata.head()

### Deal with string

In [11]:
# gender
gender_map = {'male': 1, 'female': 0}
pdata['gender'] = pdata['gender'].map(gender_map)

# party
party_map = {'KMT': 1, 'Democratic Progressive Party': 2, 'none': 3}
pdata['party'] = pdata['party'].map(party_map)

pdata.head()

,gender,age,party,willToReunification,hasMCTfamily,loveChina,loveFerrisWheel,fanOfHan
0,0,31,2,0.04,0,0,0,0
1,1,58,2,0.98,0,0,1,1
2,0,68,2,0.35,0,0,0,1
3,0,77,3,0.48,0,0,1,1
4,0,36,1,0.75,0,0,1,1


### Seperate to training set and testing set

In [12]:
pdata_train = pdata[:15000]
pdata_test = pdata[15000:]

### Seperate attributes and answer

In [13]:
y = pdata_train['fanOfHan'].values
pdata_train = pdata_train.drop('fanOfHan', axis=1)

### Draw Decision Tree

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydotplus

dtree=DecisionTreeClassifier(max_depth=4)
dtree.fit(pdata_train, y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(pdata_train),
                class_names=['Chi Mai','Mayor Han'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
# graph.write_pdf("tree.pdf")
graph.write_png("tree.png")

True

<img src="tree.png" width="900" />

## 5. Compare with rules and then calculate accuracy

In [15]:
y_test = pdata_test['fanOfHan'].values
X_test = pdata_test.drop('fanOfHan', axis=1)

y_predict = dtree.predict(X_test)

y_predict

array([0, 1, 0, ..., 1, 1, 1])

In [16]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.9534

## 6. Futher Discussion
### If we add some noise to the dataset, what will the tree be like?

In [17]:
test_data = pdata.copy()
idx = list(random.randint(0, 20000-1) for x in range(1, 50))
print(idx)
for i in idx:
    test_data.loc[i] = [
        random.randint(0, 1),
        random.randint(20, 90),
        random.randint(1, 3),
        random.uniform(0, 1),
        random.randint(0, 1),
        random.randint(0, 1),
        random.randint(0, 1),
        random.randint(0, 1)
    ]

test_data_train = test_data[:15000]
test_data_test = test_data[15000:]
test_y = test_data_train['fanOfHan'].values
test_data_train = test_data_train.drop('fanOfHan', axis=1)
test_data_train.head()

[18017, 10922, 11176, 14529, 11426, 9655, 15047, 8442, 13303, 19860, 14225, 1646, 15343, 19160, 12638, 13237, 11183, 5700, 18951, 3614, 10464, 2395, 17058, 13596, 1642, 2915, 6624, 16380, 7378, 2619, 14297, 18158, 4906, 18037, 12811, 986, 4770, 17544, 3875, 7827, 6887, 6741, 6580, 13239, 1742, 11194, 17723, 16407, 7479]


,gender,age,party,willToReunification,hasMCTfamily,loveChina,loveFerrisWheel
0,0,31,2,0.04,0,0,0
1,1,58,2,0.98,0,0,1
2,0,68,2,0.35,0,0,0
3,0,77,3,0.48,0,0,1
4,0,36,1,0.75,0,0,1


In [18]:
dtree.fit(test_data_train, y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(test_data_train),
                class_names=['Chi Mai','Mayor Han'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png("data/test_tree.png")

True

<img src="data/test_tree.png" width="900" />

The tree became a mess.

It shows that Decision Tree is prone to noise, even though only 0.25% of dataset is randomly made.

### What about the accuracy?

In [19]:
y_test = test_data_test['fanOfHan'].values
X_test = test_data_test.drop('fanOfHan', axis=1)

y_predict = dtree.predict(X_test)

accuracy_score(y_test, y_predict)

0.9522

Seems like the accuracy isn't affected that much. The tree is still well functioning whle it looks complicated.